In [1]:
import requests
from IPython.display import Markdown
from tqdm import tqdm, tqdm_notebook
import pandas as pd
from matplotlib import pyplot as plt
import altair as alt
from requests.utils import quote
import os
from github import Github

In [2]:
end_date = "2019-06-01"
start_date = "2019-05-01"
renderer = "jupyterlab"

In [3]:
# Parameters
end_date = "2019-06-25"
start_date = "2019-05-26"
renderer = "kaggle"


In [4]:
# Create the renderer we want
alt.renderers.enable(renderer)

RendererRegistry.enable('kaggle')

In [5]:
# Discourse API key
api = {'Api-Key': os.environ['DISCOURSE_API_KEY'],
       'Api-Username': os.environ['DISCOURSE_API_USERNAME']}

In [6]:
# Discourse
def topics_to_markdown(topics, n_list=10):
    body = []
    for _, topic in topics.iterrows():
        title = topic['fancy_title']
        slug = topic['slug']
        posts_count = topic['posts_count']
        url = f'https://discourse.jupyter.org/t/{slug}'
        body.append(f'* [{title}]({url}) ({posts_count} posts)')
    body = body[:n_list]
    return '\n'.join(body)

def counts_from_activity(activity):
    counts = activity.groupby('category_id').count()['bookmarked'].reset_index()
    counts['parent_category'] = None
    for ii, irow in counts.iterrows():
        if parent_categories[irow['category_id']] is not None:
            counts.loc[ii, 'parent_category'] = parent_categories[irow['category_id']]

    counts['category_id'] = counts['category_id'].map(lambda a: category_mapping[a])
    counts['parent_category'] = counts['parent_category'].map(lambda a: category_mapping[a] if a is not None else 'parent')
    is_parent = counts['parent_category'] == 'parent'
    counts.loc[is_parent, 'parent_category'] = counts.loc[is_parent, 'category_id'] 
    counts['parent/category'] = counts.apply(lambda a: a['parent_category']+'/'+a['category_id'], axis=1)
    counts = counts.sort_values(['parent_category', 'bookmarked'], ascending=False)
    return counts

# GitHub
def run_github_query(query, github_base="https://api.github.com/search/issues"):
    gh = Github(os.environ["GITHUB_ACCESS_TOKEN"])
    resp = gh.search_issues(query)
    data = []
    for ii in range(0, 10):
        items = resp.get_page(ii)
        if len(items) > 0:
            items_list = [item.raw_data for item in items]
            data.append(pd.DataFrame.from_records(items_list))
        else:
            break
    if len(data) == 0:
        return pd.DataFrame([])
    else:
        return pd.concat(data)
    


# Community forum activity

The [Jupyter Community Forum](https://discourse.jupyter.org) is a place for Jovyans across the
community to talk about Jupyter tools in interactive computing and how they fit into their
workflows. It's also a place for developers to share ideas, tools, tips, and help one another.

Below are a few updates from activity in the Discourse. For more detailed information about
the activity on the Community Forum, check out these links:

* [The users page](https://discourse.jupyter.org/u) has information about user activity
* [The top posts page](https://discourse.jupyter.org/top) contains a list of top posts, sorted
  by various metrics.

In [7]:
# Get categories for IDs
url = "https://discourse.jupyter.org/site.json"
resp = requests.get(url, headers=api)
category_mapping = {cat['id']: cat['name'] for cat in resp.json()['categories']}
parent_categories = {cat['id']: cat.get("parent_category_id", None) for cat in resp.json()['categories']}

In [8]:
# Base URL to use
url = "https://discourse.jupyter.org/latest.json"

## Topics with lots of likes

"Likes" are a way for community members to say thanks for a helpful post, show their
support for an idea, or generally to share a little positivity with somebody else.
These are topics that have generated lots of likes in recent history.

In [9]:
params = {"order": "likes", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most likes in recent history
liked = pd.DataFrame(resp.json()['topic_list']['topics'])
Markdown(topics_to_markdown(liked))

* [Introduce yourself!](https://discourse.jupyter.org/t/introduce-yourself) (93 posts)
* [{WIP} Re-Design of Jupyter.org Website](https://discourse.jupyter.org/t/wip-re-design-of-jupyter-org-website) (30 posts)
* [Jupyter Community Calls](https://discourse.jupyter.org/t/jupyter-community-calls) (28 posts)
* [Help us choose an updated &ldquo;Launch Binder&rdquo; badge!](https://discourse.jupyter.org/t/help-us-choose-an-updated-launch-binder-badge) (21 posts)
* [PyCon 2019 and mybinder.org](https://discourse.jupyter.org/t/pycon-2019-and-mybinder-org) (17 posts)
* [The Binder Federation](https://discourse.jupyter.org/t/the-binder-federation) (14 posts)
* [Jupyter community workshop: proposal on diversity, inclusion and accessibility](https://discourse.jupyter.org/t/jupyter-community-workshop-proposal-on-diversity-inclusion-and-accessibility) (20 posts)
* [Sunset the GitHub repo jupyter/help?](https://discourse.jupyter.org/t/sunset-the-github-repo-jupyter-help) (17 posts)
* [Reviving the all-Jupyter community meetings](https://discourse.jupyter.org/t/reviving-the-all-jupyter-community-meetings) (17 posts)
* [Binder/BinderHub Workshops &ndash; Help Wanted!](https://discourse.jupyter.org/t/binder-binderhub-workshops-help-wanted) (25 posts)

## Active topics on Discourse

These are topics with lots of activity in recent history.

In [10]:
params = {"order": "posts", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most posts in recent history
posts = pd.DataFrame(resp.json()['topic_list']['topics'])
Markdown(topics_to_markdown(posts))

* [Introduce yourself!](https://discourse.jupyter.org/t/introduce-yourself) (93 posts)
* [Tip: embed custom github content in a Binder link with nbgitpuller](https://discourse.jupyter.org/t/tip-embed-custom-github-content-in-a-binder-link-with-nbgitpuller) (37 posts)
* [{WIP} Re-Design of Jupyter.org Website](https://discourse.jupyter.org/t/wip-re-design-of-jupyter-org-website) (30 posts)
* [Would a &ldquo;The Littlest Binder&rdquo; be useful?](https://discourse.jupyter.org/t/would-a-the-littlest-binder-be-useful) (30 posts)
* [Plugins of considerations](https://discourse.jupyter.org/t/plugins-of-considerations) (28 posts)
* [Jupyter Community Calls](https://discourse.jupyter.org/t/jupyter-community-calls) (28 posts)
* [A proposal for JupyterHub communications](https://discourse.jupyter.org/t/a-proposal-for-jupyterhub-communications) (26 posts)
* [Binder/BinderHub Workshops &ndash; Help Wanted!](https://discourse.jupyter.org/t/binder-binderhub-workshops-help-wanted) (25 posts)
* [Help us choose an updated &ldquo;Launch Binder&rdquo; badge!](https://discourse.jupyter.org/t/help-us-choose-an-updated-launch-binder-badge) (21 posts)
* [Potential collaboration on user research?](https://discourse.jupyter.org/t/potential-collaboration-on-user-research) (21 posts)

In [11]:
counts = counts_from_activity(posts)
alt.Chart(data=counts, width=700, height=300, title="Activity by category").mark_bar().encode(
    x=alt.X("parent/category", sort=alt.Sort(counts['category_id'].values.tolist())),
    y="bookmarked",
    color="parent_category"
)

Chart({
  data:            category_id  bookmarked     parent_category  \
  9       Admin requests           1       Site Feedback   
  0                 Meta           5                Meta   
  1        announcements           1                Meta   
  2           JupyterHub           2          JupyterHub   
  6              discuss           2          JupyterHub   
  8   Enterprise Gateway           1  Enterprise Gateway   
  3       Community Chat           4      Community Chat   
  4               Binder           6              Binder   
  7              discuss           6              Binder   
  5            BinderHub           1              Binder   
  10    mybinder.org ops           1              Binder   
  
                            parent/category  
  9            Site Feedback/Admin requests  
  0                               Meta/Meta  
  1                      Meta/announcements  
  2                   JupyterHub/JupyterHub  
  6                      JupyterHub/discuss  
  8   Enterprise Gateway/Enterprise Gateway  
  3           Community Chat/Community Chat  
  4                           Binder/Binder  
  7                          Binder/discuss  
  5                        Binder/BinderHub  
  10                Binder/mybinder.org ops  ,
  encoding: EncodingWithFacet({
    color: Color({
      shorthand: 'parent_category'
    }),
    x: X({
      shorthand: 'parent/category',
      sort: Sort(['Admin requests', 'Meta', 'announcements', 'JupyterHub', 'discuss', 'Enterprise Gateway', 'Community Chat', 'Binder', 'discuss', 'BinderHub', 'mybinder.org ops'])
    }),
    y: Y({
      shorthand: 'bookmarked'
    })
  }),
  height: 300,
  mark: 'bar',
  title: 'Activity by category',
  width: 700
})

## Recently-created topics

These are topics that were recently created, sorted by the amount of activity
in each one.

In [12]:
params = {"order": "created", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Sort created by the most posted for recently-created posts
created = pd.DataFrame(resp.json()['topic_list']['topics'])
created = created.sort_values('posts_count', ascending=False)
Markdown(topics_to_markdown(created))

* [Internal Server Error with notebook with conda dependencies](https://discourse.jupyter.org/t/internal-server-error-with-notebook-with-conda-dependencies) (10 posts)
* [I get `Read timed out` errors on when pushing image OVH](https://discourse.jupyter.org/t/i-get-read-timed-out-errors-on-when-pushing-image-ovh) (8 posts)
* [JupyterLab 1.0 Release Candidate published](https://discourse.jupyter.org/t/jupyterlab-1-0-release-candidate-published) (8 posts)
* [JupyterHub, Binder workshop](https://discourse.jupyter.org/t/jupyterhub-binder-workshop) (8 posts)
* [GenericAuthenticator Redirect Issue](https://discourse.jupyter.org/t/genericauthenticator-redirect-issue) (5 posts)
* [One-click JupyterHub K8s deployment on Gitlab](https://discourse.jupyter.org/t/one-click-jupyterhub-k8s-deployment-on-gitlab) (4 posts)
* [Quick question on jupyterlab doc regarding &lt;lab-location&gt;](https://discourse.jupyter.org/t/quick-question-on-jupyterlab-doc-regarding-lab-location) (4 posts)
* [0.5 G for memory limit?](https://discourse.jupyter.org/t/0-5-g-for-memory-limit) (4 posts)
* [Welcoming our new JupyterHub team member: Sarah!](https://discourse.jupyter.org/t/welcoming-our-new-jupyterhub-team-member-sarah) (3 posts)
* [Three-dot continuation in Python Notebook](https://discourse.jupyter.org/t/three-dot-continuation-in-python-notebook) (3 posts)

In [13]:
counts = counts_from_activity(created)
alt.Chart(data=counts, width=700, height=300, title="Activity by category").mark_bar().encode(
    x=alt.X("parent/category", sort=alt.Sort(counts['category_id'].values.tolist())),
    y="bookmarked",
    color="parent_category"
)

Chart({
  data:                          category_id  bookmarked parent_category  \
  9                                Q&A           5             Q&A   
  1                               Meta           1            Meta   
  8                         JupyterLab           3      JupyterLab   
  0   Zero to JupyterHub on Kubernetes           3      JupyterHub   
  10                              team           2      JupyterHub   
  2                         JupyterHub           1      JupyterHub   
  6                            discuss           1      JupyterHub   
  5            The Littlest JupyterHub           0      JupyterHub   
  3                     Community Chat           0  Community Chat   
  4                             Binder           2          Binder   
  7                            discuss           1          Binder   
  
                                  parent/category  
  9                                       Q&A/Q&A  
  1                                     Meta/Meta  
  8                         JupyterLab/JupyterLab  
  0   JupyterHub/Zero to JupyterHub on Kubernetes  
  10                              JupyterHub/team  
  2                         JupyterHub/JupyterHub  
  6                            JupyterHub/discuss  
  5            JupyterHub/The Littlest JupyterHub  
  3                 Community Chat/Community Chat  
  4                                 Binder/Binder  
  7                                Binder/discuss  ,
  encoding: EncodingWithFacet({
    color: Color({
      shorthand: 'parent_category'
    }),
    x: X({
      shorthand: 'parent/category',
      sort: Sort(['Q&A', 'Meta', 'JupyterLab', 'Zero to JupyterHub on Kubernetes', 'team', 'JupyterHub', 'discuss', 'The Littlest JupyterHub', 'Community Chat', 'Binder', 'discuss'])
    }),
    y: Y({
      shorthand: 'bookmarked'
    })
  }),
  height: 300,
  mark: 'bar',
  title: 'Activity by category',
  width: 700
})

# GitHub activity

Jupyter also has lots of activity across GitHub repositories. The following sections contain
overviews of recent activity across the following GitHub organizations:

* [github.com/jupyterhub](https://github.com/jupyterhub)
* [github.com/jupyter](https://github.com/jupyter)
* [github.com/jupyterlab](https://github.com/jupyterlab)
* [github.com/jupyter-widgets](https://github.com/jupyter-widgets)

In [14]:
from datetime import timedelta

orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets", "ipython", "binder-examples"]
Markdown(f"Showing GitHub activity from **{start_date}** to **{end_date}**")

Showing GitHub activity from **2019-05-26** to **2019-06-25**

## Merged Pull requests

Here's an analysis of **merged pull requests** across each of the repositories in the Jupyter
ecosystem.

In [15]:
# Update data
merged = []
for org in tqdm(orgs):
    query = f"is:merged user:{org} archived:false closed:{start_date}..{end_date}"
    this_merged = run_github_query(query)
    this_merged['org'] = org
    merged.append(this_merged)
merged = pd.concat(merged)
merged['repo'] = merged['repository_url'].map(lambda a: a.split('/')[-1])

# Count PRs by org/repo
prs_by_repo = merged.groupby(['org', 'repo']).count()['body'].reset_index().sort_values(['org', 'body'], ascending=False)

100%|██████████| 6/6 [02:36<00:00, 20.11s/it]


In [16]:
alt.Chart(data=prs_by_repo, title="Merged PRs in the last 30 days").mark_bar().encode(
    x=alt.X('repo', sort=prs_by_repo['repo'].values.tolist()),
    y='body',
    color='org'
)

Chart({
  data:                 org                         repo  body
  41       jupyterlab                   jupyterlab   105
  45       jupyterlab               jupyterlab-git     7
  51       jupyterlab            jupyterlab_server     6
  43       jupyterlab     jupyterlab-data-explorer     3
  44       jupyterlab              jupyterlab-demo     3
  42       jupyterlab        jupyterlab-commenting     2
  50       jupyterlab               jupyterlab-toc     2
  40       jupyterlab            jupyter-renderers     1
  46       jupyterlab            jupyterlab-github     1
  47       jupyterlab             jupyterlab-latex     1
  48       jupyterlab  jupyterlab-metadata-service     1
  49       jupyterlab         jupyterlab-telemetry     1
  32       jupyterhub          mybinder.org-deploy    51
  36       jupyterhub      the-littlest-jupyterhub    22
  29       jupyterhub                    binderhub    14
  27       jupyterhub         alabaster-jupyterhub     9
  28       jupyterhub                 batchspawner     7
  31       jupyterhub                   jupyterhub     7
  39       jupyterhub       zero-to-jupyterhub-k8s     5
  35       jupyterhub                 team-compass     4
  30       jupyterhub                   chartpress     2
  33       jupyterhub                  nbgitpuller     2
  38       jupyterhub                  wrapspawner     2
  34       jupyterhub               systemdspawner     1
  37       jupyterhub                traefik-proxy     1
  21  jupyter-widgets                   ipywidgets    16
  25  jupyter-widgets                     tutorial    16
  20  jupyter-widgets                   ipyleaflet    10
  22  jupyter-widgets               jupyter-sphinx     4
  23  jupyter-widgets                    pythreejs     2
  24  jupyter-widgets                   traittypes     1
  26  jupyter-widgets          widget-cookiecutter     1
  14          jupyter                     nbgrader    42
  16          jupyter                     notebook    24
  5           jupyter           enterprise_gateway    14
  18          jupyter                  repo2docker    12
  12          jupyter                    nbconvert    10
  9           jupyter                 jupyter_core     7
  7           jupyter                 jupyter-book     6
  19          jupyter                    telemetry     4
  17          jupyter                    qtconsole     2
  4           jupyter                docker-stacks     1
  6           jupyter                      jupyter     1
  8           jupyter               jupyter_client     1
  10          jupyter               jupyter_server     1
  11          jupyter                        nb2kg     1
  13          jupyter                       nbdime     1
  15          jupyter                     nbviewer     1
  2           ipython                      ipython    35
  1           ipython                    ipykernel     2
  3           ipython                    traitlets     2
  0   binder-examples                        latex     1,
  encoding: EncodingWithFacet({
    color: Color({
      shorthand: 'org'
    }),
    x: X({
      shorthand: 'repo',
      sort: ['jupyterlab', 'jupyterlab-git', 'jupyterlab_server', 'jupyterlab-data-explorer', 'jupyterlab-demo', 'jupyterlab-commenting', 'jupyterlab-toc', 'jupyter-renderers', 'jupyterlab-github', 'jupyterlab-latex', 'jupyterlab-metadata-service', 'jupyterlab-telemetry', 'mybinder.org-deploy', 'the-littlest-jupyterhub', 'binderhub', 'alabaster-jupyterhub', 'batchspawner', 'jupyterhub', 'zero-to-jupyterhub-k8s', 'team-compass', 'chartpress', 'nbgitpuller', 'wrapspawner', 'systemdspawner', 'traefik-proxy', 'ipywidgets', 'tutorial', 'ipyleaflet', 'jupyter-sphinx', 'pythreejs', 'traittypes', 'widget-cookiecutter', 'nbgrader', 'notebook', 'enterprise_gateway', 'repo2docker', 'nbconvert', 'jupyter_core', 'jupyter-book', 'telemetry', 'qtconsole', 'docker-stacks', 'jupyter', 'jupyter_client', 'jupyter_server', 'nb2kg', 'nbdime', 'nbviewer', 

### A list of merged PRs by project

In [17]:
from ipywidgets import widgets, HTML
from ipywidgets import HTML
from markdown import markdown 

In [18]:
tabs = widgets.Tab(children=[])
for ii, (org, idata) in enumerate(merged.groupby('org')):
    issue_md = []
    issue_md.append(f"#### Closed PRs for org: `{org}`")
    issue_md.append("")
    for repourl, prs in idata.groupby('repository_url'):
        reponame, repo = repourl.split('/')[-2:]
        issue_md.append(f"##### [{reponame}/{repo}](https://github.com/{reponame}/{repo})")
        for _, pr in prs.iterrows():
            user_name = pr['user']['login']
            user_url = pr['user']['html_url']
            pr_number = pr['number']
            pr_html = pr['html_url']
            pr_title = pr['title']
            pr_closedby = pr['closed_by']['login']
            pr_closedby_url = pr['closed_by']['html_url']
            text = f"* [(#{pr_number})]({pr_html}): _{pr_title}_ by **[@{user_name}]({user_url})** merged by **[@{pr_closedby}]({pr_closedby_url})**"
            issue_md.append(text)
    issue_md.append('')
    markdown_html = markdown('\n'.join(issue_md))
    
    children = list(tabs.children)
    children.append(HTML(markdown_html))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
tabs

## Issues

Issues are **conversations** that happen on our GitHub repositories. Here's an
analysis of issues across the Jupyter organizations.

In [19]:
# Created issues
created = []
for org in tqdm(orgs):
    query = f"is:open is:issue user:{org} archived:false created:{start_date}..{end_date}"
    this_created = run_github_query(query)
    if len(this_created) == 0:
        continue
    this_created['org'] = org
    created.append(this_created)
created = pd.concat(created)
created['repo'] = created['repository_url'].map(lambda a: a.split('/')[-1])

100%|██████████| 6/6 [01:42<00:00, 12.57s/it]


In [20]:
created_counts = created.groupby(['org', 'repo']).count()['number'].reset_index()
created_counts['org/repo'] = created_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = created_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=created_counts, title="Issues created in the last 30 days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                       repo  number  \
  0   binder-examples               jupyter-rise       1   
  1           ipython                  ipykernel       4   
  2           ipython                ipyparallel       1   
  3           ipython                    ipython      13   
  4           jupyter                     design       1   
  5           jupyter              docker-stacks      10   
  6           jupyter         enterprise_gateway       4   
  7           jupyter                    jupyter       2   
  8           jupyter               jupyter-book       7   
  9           jupyter             jupyter_client       5   
  10          jupyter               jupyter_core       1   
  11          jupyter        jupyter_kernel_test       1   
  12          jupyter             jupyter_server       6   
  13          jupyter                  nbconvert       5   
  14          jupyter                     nbdime       2   
  15          jupyter                   nbgrader      20   
  16          jupyter                   nbviewer       4   
  17          jupyter                   notebook      31   
  18          jupyter                  qtconsole       1   
  19          jupyter                repo2docker       7   
  20  jupyter-widgets                 ipyleaflet       5   
  21  jupyter-widgets                 ipywidgets      12   
  22  jupyter-widgets             jupyter-sphinx       4   
  23  jupyter-widgets         jupyterlab-sidecar       1   
  24  jupyter-widgets                  pythreejs       5   
  25  jupyter-widgets                   tutorial       2   
  26       jupyterhub       alabaster-jupyterhub       1   
  27       jupyterhub               batchspawner       1   
  28       jupyterhub                  binderhub       6   
  29       jupyterhub                 chartpress       2   
  30       jupyterhub    configurable-http-proxy       2   
  31       jupyterhub              dockerspawner       4   
  32       jupyterhub      firstuseauthenticator       1   
  33       jupyterhub     jupyter-rsession-proxy       1   
  34       jupyterhub                 jupyterhub      19   
  35       jupyterhub   jupyterhub-deploy-docker       2   
  36       jupyterhub             jupyterlab-hub       3   
  37       jupyterhub                kubespawner       3   
  38       jupyterhub          ldapauthenticator       2   
  39       jupyterhub           ltiauthenticator       1   
  40       jupyterhub        mybinder.org-deploy       8   
  41       jupyterhub                nbgitpuller       4   
  42       jupyterhub             oauthenticator       2   
  43       jupyterhub               team-compass       8   
  44       jupyterhub    the-littlest-jupyterhub      10   
  45       jupyterhub              traefik-proxy       1   
  46       jupyterhub                wrapspawner       2   
  47       jupyterhub     zero-to-jupyterhub-k8s      11   
  48       jupyterlab  extension-cookiecutter-ts       1   
  49       jupyterlab                 jupyterlab      88   
  50       jupyterlab   jupyterlab-data-explorer       9   
  51       jupyterlab             jupyterlab-git       2   
  52       jupyterlab    jupyterlab-google-drive       1   
  53       jupyterlab       jupyterlab-telemetry       2   
  54       jupyterlab               team-compass       1   
  
                                  org/repo  
  0           binder-examples/jupyter-rise  
  1                      ipython/ipykernel  
  2                    ipython/ipyparallel  
  3                        ipython/ipython  
  4                         jupyter/design  
  5                  jupyter/docker-stacks  
  6             jupyter/enterprise_gateway  
  7                        jupyter/jupyter  
  8                   jupyter/jupyter-book  
  9                 jupyter/jupyter_client  
  10                  jupyter/jupyter_core  
  11           jupyter/jupyter_kernel_test  
  12                jupyter/jupyter_se

In [21]:
# Closed issues
closed = []
for org in tqdm(orgs):
    query = f"is:issue user:{org} closed:{start_date}..{end_date}"
    this_closed = run_github_query(query)
    this_closed['org'] = org
    closed.append(this_closed)
closed = pd.concat(closed)
closed['repo'] = closed['repository_url'].map(lambda a: a.split('/')[-1])

100%|██████████| 6/6 [01:36<00:00, 10.74s/it]


In [22]:
closed_counts = closed.groupby(['org', 'repo']).count()['number'].reset_index()
closed_counts['org/repo'] = closed_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = closed_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=closed_counts, title="Issues closed in the last 30 days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                      repo  number  \
  0   binder-examples                     bokeh       1   
  1           ipython                   ipython      15   
  2           ipython                 traitlets       1   
  3           jupyter             docker-stacks       5   
  4           jupyter        enterprise_gateway      12   
  5           jupyter                   jupyter       1   
  6           jupyter              jupyter-book       5   
  7           jupyter              jupyter_core       9   
  8           jupyter            jupyter_server       2   
  9           jupyter                     nb2kg       1   
  10          jupyter                 nbconvert      15   
  11          jupyter                    nbdime       1   
  12          jupyter                  nbgrader      37   
  13          jupyter                  notebook      31   
  14          jupyter                 qtconsole       1   
  15          jupyter               repo2docker       4   
  16          jupyter                 terminado       1   
  17  jupyter-widgets                ipyleaflet       6   
  18  jupyter-widgets                ipywidgets      11   
  19  jupyter-widgets            jupyter-sphinx       2   
  20  jupyter-widgets                 pythreejs       1   
  21  jupyter-widgets                  tutorial       2   
  22       jupyterhub      alabaster-jupyterhub       1   
  23       jupyterhub              batchspawner       1   
  24       jupyterhub                 binderhub       4   
  25       jupyterhub             dockerspawner       1   
  26       jupyterhub                jupyterhub      14   
  27       jupyterhub       mybinder.org-deploy       6   
  28       jupyterhub               nbgitpuller       1   
  29       jupyterhub              team-compass       4   
  30       jupyterhub   the-littlest-jupyterhub      18   
  31       jupyterhub             traefik-proxy       1   
  32       jupyterhub    zero-to-jupyterhub-k8s       5   
  33       jupyterlab                jupyterlab     116   
  34       jupyterlab  jupyterlab-data-explorer       7   
  35       jupyterlab           jupyterlab-demo       2   
  36       jupyterlab            jupyterlab-git       8   
  37       jupyterlab         jupyterlab_server       1   
  
                                 org/repo  
  0                 binder-examples/bokeh  
  1                       ipython/ipython  
  2                     ipython/traitlets  
  3                 jupyter/docker-stacks  
  4            jupyter/enterprise_gateway  
  5                       jupyter/jupyter  
  6                  jupyter/jupyter-book  
  7                  jupyter/jupyter_core  
  8                jupyter/jupyter_server  
  9                         jupyter/nb2kg  
  10                    jupyter/nbconvert  
  11                       jupyter/nbdime  
  12                     jupyter/nbgrader  
  13                     jupyter/notebook  
  14                    jupyter/qtconsole  
  15                  jupyter/repo2docker  
  16                    jupyter/terminado  
  17           jupyter-widgets/ipyleaflet  
  18           jupyter-widgets/ipywidgets  
  19       jupyter-widgets/jupyter-sphinx  
  20            jupyter-widgets/pythreejs  
  21             jupyter-widgets/tutorial  
  22      jupyterhub/alabaster-jupyterhub  
  23              jupyterhub/batchspawner  
  24                 jupyterhub/binderhub  
  25             jupyterhub/dockerspawner  
  26                jupyterhub/jupyterhub  
  27       jupyterhub/mybinder.org-deploy  
  28               jupyterhub/nbgitpuller  
  29              jupyterhub/team-compass  
  30   jupyterhub/the-littlest-jupyterhub  
  31             jupyterhub/traefik-proxy  
  32    jupyterhub/zero-to-jupyterhub-k8s  
  33                jupyterlab/jupyterlab  
  34  jupyterlab/jupyterlab-data-explorer  
  35           jupyterlab/jupyterlab-demo  
  36            jupyterlab/jupyterlab-git  
  37      

### A list of recent issues

In [23]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, (org, idata) in enumerate(created.groupby('org')):
    issue_md = []
    issue_md.append(f"#### {org}")
    issue_md.append("")
    for repourl, issues in idata.groupby('repository_url'):
        reponame, repo = repourl.split('/')[-2:]
        issue_md.append(f"##### [{reponame}/{repo}](https://github.com/{reponame}/{repo})")
        for _, issue in issues.sort_values('comments', ascending=False).head(n_plot).iterrows():
            user_name = issue['user']['login']
            user_url = issue['user']['html_url']
            issue_number = issue['number']
            issue_html = issue['html_url']
            issue_title = issue['title']

            text = f"* [(#{issue_number})]({issue_html}): _{issue_title}_ by **[{user_name}]({user_url})**"
            issue_md.append(text)
    issue_md.append('')
    md_html = HTML(markdown('\n'.join(issue_md)))
    
    children = list(tabs.children)
    children.append(HTML(markdown('\n'.join(issue_md))))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"Here are the top {n_plot} active issues in each repository in the last 30 days"))
display(tabs)

Here are the top 5 active issues in each repository in the last 30 days

In [24]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>